### Import Libraries

In [4]:
import numpy as np
import pandas as pd
# from absl import flags
import tensorflow as tf
# import tensorflow_privacy
# from keras.models import Model
import matplotlib.pyplot as plt
# import tensorflow_addons as tfa
# from keras.layers.convolutional import Conv2D
from tensorflow.keras.datasets import cifar10
# from tensorflow_addons.layers import GroupNormalization
# from keras.layers import Input, Dense, Flatten, Dropout, Add
# from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
# from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
# from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
# from keras.layers.pooling import MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
tf.compat.v1.disable_v2_behavior()
tf.get_logger().setLevel('ERROR')
%matplotlib inline

Instructions for updating:
non-resource variables are not supported in the long term


### Load Dataset

In [5]:
# Loading cifar10 image dataset
data_train, data_test = cifar10.load_data()
x_train, y_train = data_train
x_test, y_test = data_test

# Normalizing pixel values of images
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flattening images in dataset
y_train = y_train.flatten().reshape(y_train.shape[0], 1)
y_test = y_test.flatten().reshape(y_test.shape[0], 1)

# One hot encoding of labels/target column
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

print("training:", x_train.shape, y_train.shape)
print("testing:", x_test.shape, y_test.shape)

training: (50000, 32, 32, 3) (50000, 10)
testing: (10000, 32, 32, 3) (10000, 10)
